In [2]:
##step 1 is to read from a txt file and load the perticular data
from langchain_community.document_loaders import TextLoader
loader= TextLoader('demoSpeech.txt')
documents= loader.load()

documents

[Document(metadata={'source': 'demoSpeech.txt'}, page_content='\nwritten by-Priyansshu\nGood morning everyone,\n\nIt is a pleasure to stand before you today and share some thoughts on the importance of innovation, collaboration, and perseverance in our journey toward success. In today’s fast-paced world, the ability to adapt and embrace new ideas is more crucial than ever. Innovation is not just about technology—it’s about finding creative solutions to everyday challenges and continuously improving the way we work and live.\n\nCollaboration is the key that unlocks our collective potential. When we come together, share our perspectives, and support one another, we create an environment where everyone can thrive. Each person brings unique strengths and experiences, and by working as a team, we can achieve results that would be impossible alone.\n\nPerseverance is what keeps us moving forward, even when faced with obstacles. It is the determination to keep trying, to learn from our mistak

In [3]:
##importing up the things from the environment as well as the llm model
import os
from dotenv import load_dotenv

load_dotenv()

##gettin the api key
os.environ['GOOGLE_API_KEY']=os.getenv('GOOGLE_API_KEY')



In [4]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader= WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                      ##giving the arguments to the bs4 parser
                      bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                          
                          class_=("post-title","post-content","post-header")
                      )
                      ))

text_documents=loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
text_documents

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [6]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(documents)

documents[:5]



[Document(metadata={'source': 'demoSpeech.txt'}, page_content='written by-Priyansshu\nGood morning everyone,\n\nIt is a pleasure to stand before you today and share some thoughts on the importance of innovation, collaboration, and perseverance in our journey toward success. In today’s fast-paced world, the ability to adapt and embrace new ideas is more crucial than ever. Innovation is not just about technology—it’s about finding creative solutions to everyday challenges and continuously improving the way we work and live.\n\nCollaboration is the key that unlocks our collective potential. When we come together, share our perspectives, and support one another, we create an environment where everyone can thrive. Each person brings unique strengths and experiences, and by working as a team, we can achieve results that would be impossible alone.'),
 Document(metadata={'source': 'demoSpeech.txt'}, page_content='Perseverance is what keeps us moving forward, even when faced with obstacles. It 

In [7]:

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

##now just make the db and pass the document along with the embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(documents[:5], embeddings)

C:\Users\ppriy\AppData\Local\Temp\ipykernel_17576\1460847680.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [8]:
## just now we will make the quyery and get the ans from this database
query="who is the writer of this"
result=db.similarity_search(query)
result[0].page_content

'written by-Priyansshu\nGood morning everyone,\n\nIt is a pleasure to stand before you today and share some thoughts on the importance of innovation, collaboration, and perseverance in our journey toward success. In today’s fast-paced world, the ability to adapt and embrace new ideas is more crucial than ever. Innovation is not just about technology—it’s about finding creative solutions to everyday challenges and continuously improving the way we work and live.\n\nCollaboration is the key that unlocks our collective potential. When we come together, share our perspectives, and support one another, we create an environment where everyone can thrive. Each person brings unique strengths and experiences, and by working as a team, we can achieve results that would be impossible alone.'

In [9]:
##over here i want to use gemini llm over here
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022C993BE900>, default_metadata=(), model_kwargs={})

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template("""
    Answer the following question based on only on the context provided context
    Think step by step providing a detailed answer.
    I will tip you $1000 is the user finds the answer helpful.
    <context>
    {context}
    </context>
    Question: {input}"""
)

In [11]:
##setting up the chain for making sure that we get the proper chain to fit out our use case
from langchain.chains.combine_documents import create_stuff_documents_chain
##this perticular chain what it does is that it first converts all the context into a prompt
##then passes that to the llm and then gets the answer

documents_chain=create_stuff_documents_chain(llm,prompt)

In [12]:
##what is retriver ?? nothing it just connects the vector database to quering side so that we can retrive the info from that database more effeiciently

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000022C8CA4AA50>, search_kwargs={})

In [13]:
##now we will be making the retirver chain 
## so what basically is happening now 
##frist we query the database throug the retriever 
##then we get the response from the database
## now that response is converted to prompt tand then sent to the llm from craeting a more genring informaitve piece of text
##thats all thats how we are chaining the retriver and as well as the stuff_documant_chain;

from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,documents_chain)

In [14]:
retrieval_chain.invoke({"input":"give me a brief summary of all the content"})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 Publisher Model `projects/generativelanguage-ga/locations/us-central1/publishers/google/models/gemini-1.5-flash-002` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised NotFound: 404 Publisher Model `projects/generativelanguage-ga/locations/us-central1/publishers/google/models/gemini-1.5-flash-002` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_ret

NotFound: 404 Publisher Model `projects/generativelanguage-ga/locations/us-central1/publishers/google/models/gemini-1.5-flash-002` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions